#### MRI-ESM2-0--- Btr

In [ ]:
import os
import jCMIP as jc
Clist = jc.readList(('CMIP6listnew'))

from netCDF4 import Dataset
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import cftime
%matplotlib inline

In [ ]:
# Determine starting file and position:
def startFile(nn,Files):
    ff = 0
    nf = len(Files)
    dims = jc.getDims(Files[ff],'time')
    nm = dims[0].size
    while nn >= nm:
        nn = nn - nm
        ff = ff + 1
        if ff == nf:
            print('all files are computed')
            nn = 0
        else:
            dims = jc.getDims(Files[ff],'time')
            nm = dims[0].size
            
    return ff, nn

In [ ]:
# Define which CMIP model, experiment and ensemble member you are using:
model = Clist['MRI-ESM2-0']
EXP   = 'ssp585'
ENS   = 'r1i1p1f1'

In [ ]:
# List all files to read in:
VFiles = model.getFiles('vo',    EXP=EXP,ENS=ENS,vtype='Omon',gtype='gn')
TFiles = model.getFiles('thetao',EXP=EXP,ENS=ENS,vtype='Omon',gtype='gn')
SFiles = model.getFiles('so',    EXP=EXP,ENS=ENS,vtype='Omon',gtype='gn')
WFiles = model.getFiles('tauuo', EXP=EXP,ENS=ENS,vtype='Omon',gtype='gn')

In [ ]:
# Find coordinates to read data in from (read in data from velocity file):

ncid = Dataset(VFiles[0],'r')
if model.Oreg:
    lon  = ncid.variables[model.Olon][:]
    lat  = ncid.variables[model.Olat][:]
    lon,lat = np.meshgrid(lon,lat)
else:
    lon  = ncid.variables[model.Olon][:,:]
    lat  = ncid.variables[model.Olat][:,:]
lev  = ncid.variables['lev'][:]
ncid.close()

# Check if there is anything that needs changing in this model:
print('OextraT  ' + str(model.OextraT))
print('Ogrid    ' + str(model.Ogrid))
print('OextraWE ' + str(model.OextraWE))
print('OflipNS  ' + str(model.OflipNS))

In [ ]:
# Read in grid information:
mesh_mask = ('/home/users/jordi/mesh_masks/' + model.name + '_mesh_mask.nc')
ncid = Dataset(mesh_mask,'r')
vmask = np.squeeze(ncid.variables['vmask'][:,:,:,:])
ncid.close()

In [ ]:
# Find index of latitude closest to 26.5N (globally):
alat = 26.5
# Average latitudes over all longitudes:
mlat = np.mean(lat,axis=1)
# Find index closest 
ilat = np.argmin(np.abs(mlat-alat))  #some models might give 2 points, when 26.5 is in the middle of two points
print(ilat)

In [ ]:
# Look at choice of latitude:
fig   = plt.figure(figsize=(10,7))
plt.plot(lon[ilat,:],lat[ilat,:])
plt.tick_params(labelsize=16)

In [ ]:
# Make longitudes between -180 and 180:
lon[np.where(lon>180)] = lon[np.where(lon>180)] - 360

fig   = plt.figure(figsize=(10,7))
plt.plot(lon[ilat,:])
plt.title('longitude',fontsize=16)
plt.tick_params(labelsize=16)

In [ ]:
# Find maximum and minimum longitude:
alon_min = -80
alon_max = -10

ilon_min = np.argmin(np.abs(lon[ilat,:]-alon_min))
ilon_max = np.argmin(np.abs(lon[ilat,:]-alon_max))

In [ ]:
fig   = plt.figure(figsize=(15,10))
plt.pcolormesh(vmask[0,:,:])
plt.colorbar()
plt.axhline(y=ilat,color='C3')
plt.axvline(x=ilon_min,color='C2')
plt.axvline(x=ilon_max,color='C0')

In [ ]:
# View mask of choosen section:
fig   = plt.figure(figsize=(15,10))
plt.pcolormesh(lon[ilat,ilon_min:ilon_max],lev,vmask[:,ilat,ilon_min:ilon_max])
plt.colorbar()
# Invert y-axis:
plt.gca().invert_yaxis()

In [ ]:
# Read in non-time-varying information:
ncid   = Dataset(VFiles[0],'r')
lev    = ncid.variables['lev'][:]
cal    = ncid.variables['time'].calendar
units  = ncid.variables['time'].units
bounds = ncid.variables['time'].bounds
ncid.close()

# Note that my mesh mask files have OextraWE already removed!
ncid = Dataset(mesh_mask,'r')
dx   = ncid.variables['dxv'][ilat,ilon_min:ilon_max]
dz   = ncid.variables['dzv'][:,ilat,ilon_min:ilon_max]
ncid.close()

In [ ]:
# Setup file to store section data in (if it doesn't exist):

outfile = ('/home/users/jordi/section26data/' + model.name + '_' + EXP + '_' + ENS + '.nc')

print(outfile)

if not os.path.isfile(outfile):
    ncid = Dataset(outfile,'w')
    # Define dimentions:
    ncid.createDimension('lon',  ilon_max-ilon_min)
    ncid.createDimension('lat',  1)
    ncid.createDimension('lev',  len(lev))
    ncid.createDimension('time', None) 
    ncid.createDimension(bounds, 2)
    ncid.createDimension('ilat', 1)            

    # Define variables:
    ncid.createVariable('lon',     'f8', ('lat','lon',))
    ncid.createVariable('lat',     'f8', ('lat','lon',))
    ncid.createVariable('ilat',    'f8', ('ilat',))
    ncid.createVariable('lev',     'f8', ('lev',))
    ncid.createVariable('time',    'f8', ('time',))
    ncid.createVariable(bounds,    'f8', ('time',bounds,))
    ncid.createVariable('dx',      'f8', ('lat','lon',))
    ncid.createVariable('dz',      'f8', ('lev','lat','lon',))   # Might want to include time axis
    ncid.createVariable('vmask',   'f8', ('lev','lat','lon',))
    ncid.createVariable('vo',      'f8', ('time','lev','lat','lon',))
    ncid.createVariable('so',      'f8', ('time','lev','lat','lon',))
    ncid.createVariable('thetao',  'f8', ('time','lev','lat','lon',))
    ncid.createVariable('tauuo',   'f8', ('time','lat','lon',))   #No depth dimension
    
    # Fill in non-time varying variables:
    ncid.variables['time'].calendar = cal
    ncid.variables['time'].units    = units
    ncid.variables['time'].bounds   = bounds
    ncid.variables['lon'].units     = 'degrees_east'
    ncid.variables['lon'][:,:]      = lon[ilat,ilon_min:ilon_max]
    ncid.variables['lat'].units     = 'degrees_north'
    ncid.variables['lat'][:,:]      = lat[ilat,ilon_min:ilon_max]
    ncid.variables['ilat'][:]       = ilat
    ncid.variables['lev'][:]        = lev
    ncid.variables['vmask'][:,:,:]  = vmask[:,ilat:ilat+1,ilon_min:ilon_max]
    ncid.variables['dx'][:,:]       = dx
    ncid.variables['dz'][:,:,:]     = dz
    
    # Close file after creating:
    ncid.close()
    
    # Set month counter to 0:
    nn = 0
else:
    # Find out how much has been computed:
    ncid = Dataset(outfile,'r')
    time = ncid.variables['time'][:]
    ncid.close()
    
    nn = len(time)

In [ ]:
# Loop through each uncomputed month:
nm = nn
# Loop through each velocity file:
nf = len(VFiles)

for ff in range(0,nf):
    print(('computing file ' + str(ff+1) + ' of ' + str(nf)))
    
    # Determine how many months of data are in the current file:
    dims = jc.getDims(VFiles[ff],'vo')
    nt = dims[0].size
    
    # Determine which months have been computed:
    if nm >= nt:
        # All months from this file have been computed, move on to next file:
        nm = nm - nt
    else:
        # Loop through and compute all the missing months:
        for mm in range(nm,nt):  # mm is month in the file
                      
            # Read in velocity & time:
            ncid = Dataset(VFiles[ff],'r')
            vo   = ncid.variables['vo'][mm,:,ilat,ilon_min:ilon_max]   #reads variable vo from VFiles with specific[time,lev,lat,lon]
            tt   = ncid.variables['time'][mm]
            tb   = ncid.variables[bounds][mm,:]
            # Get Calendar information:
            units2 = ncid.variables['time'].units
            ncid.close()
            
            # Read in temperature:
            nfile, ntime = startFile(nn,TFiles) # Get indices
            ncid = Dataset(TFiles[nfile],'r') #nfile instead of ff and ntime instead of mm
            # Obtain square of T values:
            thetao1 = ncid.variables['thetao'][ntime,:,ilat,ilon_min:ilon_max]   #reads variable thetao with shape (50,70) = (depth,lon)
            thetao2 = ncid.variables['thetao'][ntime,:,ilat+1,ilon_min:ilon_max]    #needed since thetao1 is a slice with 1 lat
            thetao3 = np.roll(thetao1,-1,axis=1)    #lower right gridpoint
            thetao4 = np.roll(thetao2,-1,axis=1)    #upper right gridpoint
            
            # Calculate average of square T, so that T and u point overlap:
            thetao = np.zeros(np.shape(thetao1))
            for i in range (len(lev)):
                for j in range (ilon_max-ilon_min):
                    thetao[i,j] = ((thetao1[i,j]) + (thetao2[i,j]) + (thetao3[i,j]) + (thetao4[i,j]))/4
                    
            ncid.close()
                                 
            # Read in salinity:
            nfile, ntime = startFile(nn,SFiles) # Get indices
            ncid = Dataset(SFiles[nfile],'r') #nfile instead of ff and ntime instead of mm
            # Obtain square of T values:
            so1 = ncid.variables['so'][ntime,:,ilat,ilon_min:ilon_max]   #reads variable so with shape (50,70) = (depth,lon)
            so2 = ncid.variables['so'][ntime,:,ilat+1,ilon_min:ilon_max]    #needed since so1 is a slice with 1 lat
            so3 = np.roll(so1,-1,axis=1)    #lower right gridpoint
            so4 = np.roll(so2,-1,axis=1)    #upper right gridpoint
            
            # Calculate average of square S, so that S and u point overlap:
            so = np.zeros(np.shape(so1))
            for i in range (len(lev)):
                for j in range (ilon_max-ilon_min):
                    so[i,j] = ((so1[i,j]) + (so2[i,j]) + (so3[i,j]) + (so4[i,j]))/4
                    
            ncid.close()
            
            # Read in wind stress: (same gridpoint as vo)
            nfile, ntime = startFile(nn,WFiles) # Get indices
            ncid  = Dataset(WFiles[nfile],'r') #nfile instead of ff and ntime instead of mm
            tauuo = ncid.variables['tauuo'][ntime,ilat,ilon_min:ilon_max]   #reads variable tauuo from WFiles with specific[time,lev,lat,lon]
            ncid.close()
            
            # Make sure time variable is the same:
            tt,tb = jc.fixTime(units,units2,cal,tt,tb)
            
            # Write to file:
            ncid = Dataset(outfile,'a')
            ncid.variables['vo'][nn,:,:,:] = vo
            ncid.variables['time'][nn] = tt
            ncid.variables[bounds][nn,:] = tb
            ncid.variables['thetao'][nn,:,:,:] = thetao
            ncid.variables['so'][nn,:,:,:] = so
            ncid.variables['tauuo'][nn,:,:] = tauuo
            ncid.close()
            
            nn = nn + 1
        nm = 0
print('Done!')